# Detailed Steps for Implementing Agent-Based Modeling (ABM) for Urban Heat Resilience in AnyLogic

This guide provides a comprehensive, step-by-step approach to implementing Agent-Based Modeling (ABM) using AnyLogic to enhance your conference paper into a journal paper. It focuses on integrating ABM into Steps 3 (Predictive Model) and 4 (Prescriptive Model) of your four-step scenario planning model for urban heat resilience in Tokyo’s Nihonbashi district. The guide covers data preparation using Python, model setup in AnyLogic, and evaluation of results, addressing the gaps in the conference paper by simulating resident behavior and infrastructure interventions.

## 1. Data Preparation Using Python

To create a realistic ABM, you need to prepare data for agents (residents and shelters), the urban environment, and heat risk dynamics. Python’s libraries like `pandas`, `geopandas`, and `osmnx` are well-suited for processing this data. Below are the detailed steps.

### 1.1 Demographic Data for Resident Agents
- **Objective**: Generate profiles for resident agents with attributes like age, health status, mobility, heat sensitivity, home location, and shelter awareness.
- **Data Source**: Use e-Stat (Japan’s statistical portal, [e-Stat](https://www.e-stat.go.jp/en)) or Tokyo government census data for Nihonbashi’s demographic information, including age distribution, household composition, and health statistics.
- **Steps**:
  1. **Download Data**: Access census data for Nihonbashi, focusing on age, health conditions (e.g., prevalence of heat-related illnesses), and mobility (e.g., disability rates).
  2. **Process with Pandas**:
     - Load the data into a `pandas` DataFrame.
     - Clean and preprocess the data, handling missing values and standardizing formats.
     - Extract distributions (e.g., percentage of elderly residents, typically 20–30% in Tokyo districts).
  3. **Generate Agent Profiles**:
     - Use random sampling to assign attributes to a population of agents (e.g., 10,000 residents, scaled to represent Nihonbashi’s population).
     - Example attributes:
       - **Age**: Sample from age distribution (e.g., 20% over 65, 50% 20–64, 30% under 20).
       - **Health Status**: Assign based on health statistics (e.g., 10% with high heat sensitivity due to chronic conditions).
       - **Mobility**: Assign walking speeds (e.g., 1 m/s for elderly, 1.5 m/s for adults) or transportation access.
       - **Heat Sensitivity**: Assign a threshold (e.g., 0–1 scale, where 1 is highly sensitive) based on age and health.
       - **Shelter Awareness**: Assign a probability (e.g., 80% know shelter locations, higher for younger agents).
     - Use `numpy.random` for sampling:
       ```python
       import pandas as pd
       import numpy as np

       # Example: Generate 10,000 resident profiles
       num_agents = 10000
   
       # Age distribution
       ages = np.random.choice(
           [15, 40, 70],  # Representative ages for young, adult, elderly
           size=num_agents,
           p=[0.3, 0.5, 0.2]  # Probabilities based on census
       )
   
       # Health status (0: low risk, 1: high risk)
       health_status = np.random.choice(
           [0, 1],
           size=num_agents,
           p=[0.9, 0.1]  # 10% high risk
       )
   
       # Mobility (walking speed in m/s)
       mobility = np.where(ages > 65, np.random.normal(1, 0.2), np.random.normal(1.5, 0.3))
   
       # Heat sensitivity (0 to 1)
       heat_sensitivity = np.where(health_status == 1, np.random.uniform(0.7, 1), np.random.uniform(0, 0.7))
   
       # Shelter awareness (0 or 1)
       shelter_awareness = np.random.choice([0, 1], size=num_agents, p=[0.2, 0.8])
   
       # Create DataFrame
       residents = pd.DataFrame({
           'agent_id': range(num_agents),
           'age': ages,
           'health_status': health_status,
           'mobility': mobility,
           'heat_sensitivity': heat_sensitivity,
           'shelter_awareness': shelter_awareness
       })
       ```
  4. **Save Data**: Export to a CSV file for AnyLogic import:
     ```python
     residents.to_csv('residents.csv', index=False)
     ```

### 1.2 Geospatial Data for Environment and Home Locations
- **Objective**: Assign home locations to residents and define the urban network for movement.
- **Data Source**: Use OpenStreetMap ([OpenStreetMap](https://www.openstreetmap.org)) or PLATEAU ([PLATEAU](https://www.mlit.go.jp/plateau/)) for building and street network data.
- **Steps**:
  1. **Download Geospatial Data**:
     - Use `osmnx` to download Nihonbashi’s street network and building footprints.
     - Example:
       ```python
       import osmnx as ox
       import geopandas as gpd

       # Download street network for Nihonbashi
       place_name = "Nihonbashi, Chuo, Tokyo, Japan"
       G = ox.graph_from_place(place_name, network_type='walk')
       ox.save_graphml(G, 'nihonbashi_network.graphml')

       # Download building footprints
       buildings = ox.geometries_from_place(place_name, tags={'building': True})
       buildings.to_file('nihonbashi_buildings.geojson')
       ```
  2. **Assign Home Locations**:
     - Filter buildings to residential types (e.g., apartments, houses).
     - Randomly assign residents to building centroids.
     - Example:
       ```python
       # Filter residential buildings
       residential = buildings[buildings['building'].isin(['apartments', 'house', 'residential'])]

       # Assign home locations
       home_locations = residential.sample(n=num_agents, replace=True)[['geometry']].reset_index(drop=True)
       residents['x'] = home_locations['geometry'].x
       residents['y'] = home_locations['geometry'].y
       residents.to_csv('residents_with_locations.csv', index=False)
       ```
  3. **Prepare Network Data**:
     - Convert the street network to a format compatible with AnyLogic (e.g., shapefile or CSV of nodes and edges).
     - Example:
       ```python
       nodes, edges = ox.graph_to_gdfs(G)
       nodes.to_file('nihonbashi_nodes.shp')
       edges.to_file('nihonbashi_edges.shp')
       ```

### 1.3 Shelter Data
- **Objective**: Define shelter locations and capacities.
- **Data Source**: Identify cooling shelters (e.g., community centers, libraries) from local government data or assume based on public buildings.
- **Steps**:
  1. **Identify Shelters**:
     - Use OpenStreetMap or local government data to locate shelters.
     - Assume capacities based on building size or standards (e.g., 100 people per shelter).
  2. **Create Shelter Dataset**:
     - Example:
       ```python
       shelters = pd.DataFrame({
           'shelter_id': [1, 2, 3],
           'name': ['Community Center', 'Library', 'School'],
           'x': [139.774, 139.776, 139.778],  # Example coordinates
           'y': [35.683, 35.685, 35.687],
           'capacity': [100, 150, 200]
       })
       shelters.to_csv('shelters.csv', index=False)
       ```

### 1.4 Heat Risk Data
- **Objective**: Model heat risk levels across Nihonbashi.
- **Data Source**: Use historical meteorological data (NOAA, Weather.gov) or simulate heat risk scenarios.
- **Steps**:
  1. **Process Historical Data**:
     - Use data from 2015–2024, as in the conference paper, classified into “High,” “Moderate,” “Low” risk levels.
     - Example:
       ```python
       # Example: Simulate heat risk grid
       heat_risk = pd.DataFrame({
           'x': np.repeat(np.linspace(139.77, 139.78, 100), 100),
           'y': np.tile(np.linspace(35.68, 35.69, 100), 100),
           'heat_risk': np.random.choice(['Low', 'Moderate', 'High'], size=10000, p=[0.5, 0.3, 0.2])
       })
       heat_risk.to_csv('heat_risk.csv', index=False)
       ```
  2. **Simulate Scenarios**:
     - Create datasets for different heatwave intensities (e.g., all “High” for extreme scenarios).

## 2. Setting Up the ABM in AnyLogic

AnyLogic is a powerful simulation tool that supports ABM with GIS integration and statecharts for modeling agent behaviors ([AnyLogic Help](https://anylogic.help/anylogic/agentbased/index.html)). Below are the steps to build the model.

### 2.1 Start a New Model
- Open AnyLogic (Professional or University edition recommended).
- Create a new model: File > New > Model.
- Select “Agent-based” as the modeling approach.
- Name the model (e.g., “NihonbashiHeatResilience”).

### 2.2 Define Agent Types
- **Resident Agent**:
  1. In the Projects view, right-click “Main” > New > Agent Type, name it “Resident”.
  2. Add parameters:
     - `age` (int), `health_status` (double, 0–1), `mobility` (double, m/s), `heat_sensitivity` (double, 0–1), `shelter_awareness` (boolean), `home_location` (Point, for GIS coordinates).
  3. Add variables:
     - `current_location` (Point), `state` (String: “AtHome”, “Evacuating”, “AtShelter”), `heat_exposure` (double, cumulative exposure).
- **Shelter Agent**:
  1. Create a new agent type named “Shelter”.
  2. Add parameters: `location` (Point), `capacity` (int).
  3. Add variables: `current_occupancy` (int).

### 2.3 Set Up the Environment
- **GIS Map**:
  1. In the Main agent, drag a “GIS Map” element from the Space Markup palette.
  2. Configure the map to use OpenStreetMap or import a custom map from PLATEAU data.
  3. Set the map’s bounding box to Nihonbashi (e.g., center at 139.775°E, 35.685°N).
- **Network**:
  1. Import the street network as a shapefile or use AnyLogic’s GIS network features.
  2. Place shelters on the map using their coordinates from the shelter dataset.

### 2.4 Populate the Model
- **Residents**:
  1. In the Main agent, create a population of Resident agents: drag a “Population” element, name it “residents”.
  2. Set the population size (e.g., 10,000).
  3. Import resident data from `residents_with_locations.csv`:
     - Go to the population’s properties, select “Database” or “CSV” import.
     - Map CSV columns to agent parameters (e.g., `x` and `y` to `home_location`).
  4. Place residents at their home locations on the GIS map.
- **Shelters**:
  1. Create a population of Shelter agents, name it “shelters”.
  2. Import shelter data from `shelters.csv`, mapping columns to parameters.
  3. Place shelters on the map.

### 2.5 Define Agent Behaviors
- **Resident Behavior (Statechart)**:
  1. In the Resident agent, add a statechart from the Agent palette.
  2. Define states:
     - `AtHome`: Initial state.
     - `Evacuating`: When deciding to evacuate.
     - `AtShelter`: When reached a shelter.
  3. Define transitions:
     - **AtHome to Evacuating**: Condition: `heat_risk > heat_sensitivity && nearestShelter().current_occupancy < nearestShelter().capacity && shelter_awareness`.
       - Action: Select the nearest shelter using GIS routing, start moving using `moveTo(nearestShelter().location)`.
     - **Evacuating to AtShelter**: Condition: `distanceTo(nearestShelter().location) < 10` (meters).
       - Action: `nearestShelter().current_occupancy++`, `state = "AtShelter"`, stop movement.
  4. Implement route selection:
     - Use AnyLogic’s GIS routing to find paths, prioritizing low heat exposure (e.g., shaded routes).
     - Example: Modify path cost to include heat risk from `heat_risk.csv`.
- **Shelter Behavior**:
  1. Shelters update `current_occupancy` when residents arrive.
  2. Optionally, model shelter cooling effects or resource constraints.

### 2.6 Implement Heat Risk Dynamics
- Import `heat_risk.csv` as a dataset in Main.
- Create a function to update heat risk levels across the map based on time and location.
- Example: Use a `Schedule` element to update heat risk every hour, interpolating values from the dataset.

### 2.7 Data Collection
- Add datasets in Main to track:
  - `evacuation_success_rate`: Percentage of residents reaching shelters.
  - `average_heat_exposure`: Mean heat exposure during evacuation.
  - `shelter_occupancy`: Occupancy levels per shelter.
  - `evacuation_time`: Average time to reach shelters.
- Use AnyLogic’s `DataSet` or `TimePlot` elements for real-time tracking.

### 2.8 Run Simulations
- **Predictive Model**:
  1. Define scenarios: e.g., “Moderate” heatwave (30% High risk areas), “Extreme” heatwave (80% High risk areas).
  2. Run simulations for 24 hours (simulated time) with a 1-minute time step.
  3. Repeat 10–20 times per scenario to account for stochasticity.
- **Prescriptive Model**:
  1. Modify the environment to include interventions:
     - Add new shelters (update `shelters.csv`).
     - Create cool corridors (modify network edges to reduce heat exposure).
     - Implement green roofs (reduce heat risk in specific areas).
  2. Run simulations with interventions, comparing to baseline.

## 3. Evaluating Results

### 3.1 Metrics
- **Evacuation Success Rate**: Percentage of residents reaching shelters.
- **Average Heat Exposure**: Mean exposure during evacuation (weighted by heat risk along paths).
- **Shelter Occupancy**: Percentage of capacity used per shelter.
- **Evacuation Time**: Average time from decision to evacuate to reaching a shelter.

### 3.2 Analysis
- **Comparison**: Compare metrics between baseline and intervention scenarios.
  - Example: Does adding a new shelter increase evacuation success by 10%?
- **Statistical Analysis**:
  - Use Python or R to perform t-tests or ANOVA on metrics across runs.
  - Example:
    ```python
    import pandas as pd
    from scipy.stats import ttest_ind

    baseline = pd.read_csv('baseline_results.csv')
    intervention = pd.read_csv('intervention_results.csv')
    t_stat, p_value = ttest_ind(baseline['evacuation_success_rate'], intervention['evacuation_success_rate'])
    print(f"P-value: {p_value}")  # Check significance
    ```
- **Visualization**:
  - Use AnyLogic’s GIS map to visualize agent trajectories.
  - Export data to create heat maps or plots in Python (e.g., with Ascertain significance with `matplotlib` or `seaborn`).

### 3.3 Validation
- **Calibration**: Adjust parameters (e.g., heat sensitivity thresholds) to match historical data or case studies (e.g., IBM Japan case from the conference paper).
- **Validation**: Compare simulation outcomes with real-world heatwave response data, if available, to ensure realism.

## 4. Example Workflow
Below is a simplified example of the Python code for data preparation and a conceptual outline of the AnyLogic model setup.

### 4.1 Python Code for Data Preparation
```python
import pandas as pd
import numpy as np
import osmnx as ox
import geopandas as gpd

# Generate resident profiles
num_agents = 10000
ages = np.random.choice([15, 40, 70], size=num_agents, p=[0.3, 0.5, 0.2])
health_status = np.random.choice([0, 1], size=num_agents, p=[0.9, 0.1])
mobility = np.where(ages > 65, np.random.normal(1, 0.2), np.random.normal(1.5, 0.3))
heat_sensitivity = np.where(health_status == 1, np.random.uniform(0.7, 1), np.random.uniform(0, 0.7))
shelter_awareness = np.random.choice([0, 1], size=num_agents, p=[0.2, 0.8])

residents = pd.DataFrame({
    'agent_id': range(num_agents),
    'age': ages,
    'health_status': health_status,
    'mobility': mobility,
    'heat_sensitivity': heat_sensitivity,
    'shelter_awareness': shelter_awareness
})

# Assign home locations
place_name = "Nihonbashi, Chuo, Tokyo, Japan"
buildings = ox.geometries_from_place(place_name, tags={'building': True})
residential = buildings[buildings['building'].isin(['apartments', 'house', 'residential'])]
home_locations = residential.sample(n=num_agents, replace=True)[['geometry']].reset_index(drop=True)
residents['x'] = home_locations['geometry'].x
residents['y'] = home_locations['geometry'].y
residents.to_csv('residents_with_locations.csv', index=False)

# Save street network
G = ox.graph_from_place(place_name, network_type='walk')
nodes, edges = ox.graph_to_gdfs(G)
nodes.to_file('nihonbashi_nodes.shp')
edges.to_file('nihonbashi_edges.shp')

# Shelter data
shelters = pd.DataFrame({
    'shelter_id': [1, 2, 3],
    'name': ['Community Center', 'Library', 'School'],
    'x': [139.774, 139.776, 139.778],
    'y': [35.683, 35.685, 35.687],
    'capacity': [100, 150, 200]
})
shelters.to_csv('shelters.csv', index=False)

# Heat risk data
heat_risk = pd.DataFrame({
    'x': np.repeat(np.linspace(139.77, 139.78, 100), 100),
    'y': np.tile(np.linspace(35.68, 35.69, 100), 100),
    'heat_risk': np.random.choice(['Low', 'Moderate', 'High'], size=10000, p=[0.5, 0.3, 0.2])
})
heat_risk.to_csv('heat_risk.csv', index=False)
```

### 4.2 AnyLogic Model Outline
- **Main Agent**:
  - GIS Map: Nihonbashi area, centered at (139.775°E, 35.685°N).
  - Populations: `residents` (10,000 Resident agents), `shelters` (3 Shelter agents).
  - Datasets: `evacuation_success_rate`, `average_heat_exposure`, `shelter_occupancy`, `evacuation_time`.
- **Resident Agent**:
  - Statechart:
    - States: `AtHome`, `Evacuating`, `AtShelter`.
    - Transitions: Based on heat risk, shelter capacity, and awareness.
    - Movement: Use `moveTo()` with GIS routing, prioritizing low heat exposure.
- **Shelter Agent**:
  - Updates `current_occupancy` when residents arrive.
- **Simulation Settings**:
  - Time step: 1 minute.
  - Duration: 24 hours.
  - Scenarios: Baseline, Moderate heatwave, Extreme heatwave, Intervention scenarios.

## 5. Table: Key Metrics for Evaluation
| **Metric**                | **Description**                                      | **Baseline Example** | **Intervention Example** |
|---------------------------|-----------------------------------------------------|----------------------|--------------------------|
| Evacuation Success Rate   | % of residents reaching shelters                    | 70%                  | 85%                      |
| Average Heat Exposure     | Mean heat exposure during evacuation (0–1 scale)    | 0.6                  | 0.4                      |
| Shelter Occupancy         | % of capacity used per shelter                      | 80%                  | 60%                      |
| Evacuation Time           | Average time to reach shelters (minutes)            | 30                   | 25                       |

## 6. Resources
- **AnyLogic Tutorials**: Refer to “AnyLogic in 3 Days” book or online courses ([Noorjax](https://noorjax.teachable.com/p/agent-based-modeling-in-anylogic)) for learning AnyLogic basics.
- **Urban Dynamics Example**: Explore the Urban Dynamics Educational Simulator on AnyLogic Cloud ([AnyLogic Cloud](https://cloud.anylogic.com/model/47990ad8-ab7c-4fc5-88e5-0e3771cb5303)) for urban ABM insights.
- **Data Sources**: e-Stat ([e-Stat](https://www.e-stat.go.jp/en)), OpenStreetMap ([OpenStreetMap](https://www.openstreetmap.org)), PLATEAU ([PLATEAU](https://www.mlit.go.jp/plateau/)).

This guide provides a robust framework to integrate ABM into your journal paper, enhancing the analysis of human behavior and infrastructure impacts for urban heat resilience.